# Cleaning Data with PySpark

Working with data is tricky - working with millions or even billions of rows is worse. Did you receive some data processing code written on a laptop with fairly pristine data? Chances are you’ve probably been put in charge of moving a basic data process from prototype to production. You may have worked with real world datasets, with missing fields, bizarre formatting, and orders of magnitude more data. Even if this is all new to you, this course helps you learn what’s needed to prepare data processes using Python with Apache Spark. You’ll learn terminology, methods, and some best practices to create a performant, maintainable, and understandable data processing platform.

## Table of Contents

- [Introduction](#intro)
- [Manipulating DataFrames in the real wold](#man)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

path = "data/dc33/"

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")
print(sc)

<SparkContext master=local appName=First App>


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('First App').getOrCreate()

---
<a id='intro'></a>

## Intro to data cleaning with Apache Spark

<img src="images/spark3_001.png" alt="" style="width: 800px;"/>

<img src="images/spark3_002.png" alt="" style="width: 800px;"/>

<img src="images/spark3_003.png" alt="" style="width: 800px;"/>

<img src="images/spark3_004.png" alt="" style="width: 800px;"/>

<img src="images/spark3_005.png" alt="" style="width: 800px;"/>

## Defining a schema

Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

- Name
- Age
- City

The Name and City columns are `StringType()` and the Age column is an `IntegerType()`.

- Import * from the pyspark.sql.types library.
- Define a new schema using the StructType method.
- Define a StructField for name, age, and city. Each field should correspond to the correct datatype and not be nullable.

In [2]:
# Import the pyspark.sql.types library
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city', StringType(), False)
])

## Immutability and lazy processing

<img src="images/spark3_006.png" alt="" style="width: 800px;"/>

<img src="images/spark3_007.png" alt="" style="width: 800px;"/>

<img src="images/spark3_008.png" alt="" style="width: 800px;"/>

<img src="images/spark3_009.png" alt="" style="width: 800px;"/>

Spark takes advantage of data immutability to efficiently share / create new data representations throughout the cluster.

## Using lazy processing

`Lazy processing operations` will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to `Spark not performing any transformations until an action is requested`.

For this exercise, we'll be defining a Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

- Load the Data Frame.
- Add the transformation for F.lower() to the Destination Airport column.
- Drop the Destination Airport column from the Data Frame aa_dfw_df. Note the time for these operations to complete.
- Show the Data Frame, noting the time difference for this action to complete.

In [10]:
from pyspark.sql import functions as F

# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load(path+'AA_DFW_2017_Departures_Short.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
|       01/01/2017|         0060|                          149|    mia|
|       01/01/2017|         0071|                          203|    lax|
|       01/01/2017|         0074|                           76|    mem|
|       01/01/2017|         0081|                          123|    den|
|       01/01/2017|         0089|                          161|    slc|
|       01/01/2017|         0096|                           84| 

You've just seen how lazy processing works in action. Remember when working with Spark that no transformations take effect until you apply an action. This can be confusing at times, but is one of the underpinnings of Spark's power.

## Understanding Parquet

<img src="images/spark3_010.png" alt="" style="width: 800px;"/>

<img src="images/spark3_011.png" alt="" style="width: 800px;"/>

<img src="images/spark3_012.png" alt="" style="width: 800px;"/>

<img src="images/spark3_013.png" alt="" style="width: 800px;"/>

<img src="images/spark3_014.png" alt="" style="width: 800px;"/>

## Saving a DataFrame in Parquet format

When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The `Parquet format` is a columnar data store, allowing Spark to use `predicate pushdown`. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

The spark object and the df1 and df2 DataFrames have been setup for you.

- View the row count of df1 and df2.
- Combine df1 and df2 in a new DataFrame named df3 with the union method.
- Save df3 to a parquet file named AA_DFW_ALL.parquet.
- Read the AA_DFW_ALL.parquet file and show the count.

In [ ]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

```
<script.py> output:
    df1 Count: 139359
    df2 Count: 119911
    259270
```

## SQL and Parquet

`Parquet files are perfect as a backing data store for SQL queries in Spark`. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

The spark object and the AA_DFW_ALL.parquet file are available for you automatically.

- Import the AA_DFW_ALL.parquet file into flights_df.
- Use the createOrReplaceTempView method to alias the flights table.
- Run a Spark SQL query against the flights table.

In [ ]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

```
<script.py> output:
    The average flight time is: 151
```
You've just run a SQL query against a Parquet data source. When building production Spark code, you'll often port SQL operations directly.

---
<a id='man'></a>

## Manipulating DataFrames in the real wold

## DataFrame column operations

<img src="images/spark3_015.png" alt="" style="width: 800px;"/>

<img src="images/spark3_016.png" alt="" style="width: 800px;"/>

<img src="images/spark3_017.png" alt="" style="width: 800px;"/>

<img src="images/spark3_018.png" alt="" style="width: 800px;"/>

<img src="images/spark3_019.png" alt="" style="width: 800px;"/>

## 

In [ ]:
<img src="images/spark3_020.png" alt="" style="width: 800px;"/>

In [ ]:
---
<a id='intro'></a>